In [ ]:
%config Completer.use_jedi = False

In [ ]:
from pymongo import MongoClient
from pymongo.cursor import Cursor

import pandas as pd
from datetime import datetime

from predictsignauxfaibles.utils import MongoDBQuery, MongoParams
import predictsignauxfaibles.config as config
import pytz

In [ ]:
date_min = "2020-02-01"
date_max = "2020-02-28"

def date_to_iso(date: str):
        """
        Converts a date in the YYYY-MM-DD format into a
        datetime usable by mongodb"
        """
        return pytz.utc.localize(datetime.strptime(date, "%Y-%m-%d"))

## Fetching features and scores data

In [ ]:
features_mongo = MongoClient(host = "mongodb://labbdd").get_database("prod").get_collection("Features")

match_stage = {
    "$match": {
        "$and": [
            {
                "_id.periode": {
                    "$gte": date_to_iso(date_min),
                    "$lt": date_to_iso(date_max),
                    },
                "_id.batch": "2103_0_urssaf"
            },
            #{"value.effectif": {"$gte": min_effectif}},
        ]
    }
}

project_stage = {
    "$project": {field: 1 for field in ["_id.siret", "_id.periode", "value.effectif", "value.outcome", "value.code_commune", "value.departement", "value.region", "value.code_ape", "value.age", "value.code_naf", "value.effectif_entreprise"]}
}
pipeline = [match_stage, project_stage]

features_cursor = features_mongo.aggregate(pipeline)

In [ ]:
features = pd.DataFrame(features_cursor)
features.to_json("/home/simon.lebastard/predictsignauxfaibles/data/features_2103.json", orient="records")

In [ ]:
feature_ids = features["_id"].apply(pd.Series)
features_values = features["value"].apply(pd.Series)
features = pd.concat([feature_ids, features_values], axis=1)


In [ ]:
features.sample(n=5)

In [ ]:
scores_mongo = MongoClient(host = "mongodb://labbdd").get_database("prod").get_collection("Scores")
scores_cursor = scores_mongo.find({"algo": "mars2021_v0"})
scores = pd.DataFrame(scores_cursor)

In [ ]:
scores.periode = scores.periode.apply(date_to_iso)

In [ ]:
scores.periode = scores.periode.apply(date_to_iso)

In [ ]:
scores.dtypes

In [ ]:
df = pd.merge(scores, features, on=['siret', 'periode'], how='inner') #join may be empty because scores.periode is a string (format "2014-01-01") and features.periode is of type Date (2014-01-01T00:00:00.000+00:00)

In [ ]:
df.sample(n=5)

## Aggregation of region-wide features

Niveaux de granularité considérés:
- région
- département

Pour chaque niveau de granularité:
- compter le nombre d'entreprises flaguées rouge par région
- compter le nombre d'entreprises flaguées orange par région
- calculer les ratio correspondants
- [BONUS: A NE PAS COMMUNIQUER] produire un score de risque moyen par région
- compter le nombre de défaillances effectives sur une période donnée, et calculer le ratio correspondants

Pour toutes les grandeurs calculées précédemment, calculer des équivalents en nombre d'employés concernés

En bonus: calculer des indicateurs de contribution des chaques 4 macrovariables du modèle

In [ ]:
df_by_region = df.groupby(by="region").count()

Pour récuperer les coefficients du modèle de mars 2021, chargeons le modèle sauvegardé sur labtenant: